# NEXUS tool: case study for the NWSAS basin - energy demand calculations
In this notebook a case study for the NWSAS basin is covered using the `nexus_tool` package. The water requirements for agricultural irrigation were previously calculated (check the water demand case study). From there, the energy requirements for pumping and desalination of brackish water are estimated.

First import the package by running the following block:

In [1]:
import nexus_tool

After importing all required packages, the input GIS data is loaded into the variable `df`. Change the `file_path` variable to reflect the name and relative location of your data file. This dataset should already have the water demand for irrigation data.

In [2]:
file_path = r'nwsas_results/water_demand.csv'
df = nexus_tool.read_csv(file_path)

## 1. Creating the model
To create a model simply create an instance of the `nexus_tool.Model()` class and store it in a variable name. The `nexus_tool.Model()` class requires a dataframe as input data. Several other properties and parameter values can be defined by explicitly passing values to them. To see a full list of parameters and their explaination refer to the documentation of the package.

In [3]:
nwsas = nexus_tool.Model(df)

### 1.1. Changing default values for inputs

### 1.2. Calculating the Total Dinamic Head (tdh)
First, we will calculate the Total Dinamic Head (tdh), which is a measure in meters that accounts for the depth to groundwater, the water drawdown (wdd), the operating application pressure (oap) and the pressure loss in distribution (pld).

For that, run the `nwsas.get_gw_tdh()` method the `wdd`, `oap` and `pld` values. Also, make sure that the name for the groundwater depth column of the dataframe, matches the name of the `gw_depth` variable in the model. For that run `nwsas.gw_depth` or the `nwsas.print_properties()` method and check that the name is ok. If you need to change it, then run something like:
```python
nwsas.gw_depth = 'name_of_ground_water_depth'
```


In [4]:
nwsas.get_gw_tdh(inplace = True)